In [7]:
# pip install duckduckgo-search
from langchain.agents.agent_toolkits import create_python_agent
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType
from langchain.tools.python.tool import PythonREPLTool
from langchain.python import PythonREPL
from langchain.chat_models import ChatOpenAI
from langchain.tools import DuckDuckGoSearchRun
from langchain.utilities import WikipediaAPIWrapper
from langchain.python import PythonREPL
from langchain.agents import Tool
from langchain.agents import initialize_agent
from langchain.chat_models import ChatOpenAI
import random
 
llm = ChatOpenAI(temperature=0)
 
wikipedia = WikipediaAPIWrapper()
search = DuckDuckGoSearchRun()
python_repl = PythonREPL()
 
 
#自定义工具，获取随机数
def random_num(input=""):
    return random.randint(0,5)
 
#定义维基百科工具
wikipedia_tool = Tool(
    name='wikipedia',
    func= wikipedia.run,
    description="Useful for when you need to look up a topic, country or person on wikipedia"
)
 
#定义duckduckgo 搜索工具
duckduckgo_tool = Tool(
    name='DuckDuckGo Search',
    func= search.run,
    description="Useful for when you need to do a search on the internet to find information that another tool can't find. \
    be specific with your input."
)
 
#定义python工具
python_tool = Tool(
    name = "python repl",
    func=python_repl.run,
    description="useful for when you need to use python to answer a question. You should input python code"
)
 
#定义获取随机数工具
random_tool = Tool(
    name='Random number',
    func= random_num,
    description="Useful for when you need to get a random number. input should be 'random'"
)
 
#整合所有工具
tools=[python_tool,wikipedia_tool,duckduckgo_tool,random_tool]
 
#创建代理
zero_shot_agent = initialize_agent(
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=5,
)

print(zero_shot_agent.agent.llm_chain.prompt.template)

Answer the following questions as best you can. You have access to the following tools:

python repl: useful for when you need to use python to answer a question. You should input python code
wikipedia: Useful for when you need to look up a topic, country or person on wikipedia
DuckDuckGo Search: Useful for when you need to do a search on the internet to find information that another tool can't find.     be specific with your input.
Random number: Useful for when you need to get a random number. input should be 'random'

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [python repl, wikipedia, DuckDuckGo Search, Random number]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

B